In [1]:
%%bash

pip install -q kaggle
mkdir -p ~/.kaggle
cp drive/MyDrive/kaggle.json ~/.kaggle/
chmod 600 /root/.kaggle/kaggle.json
kaggle datasets download -d gpiosenka/100-bird-species

100%|██████████| 1.69G/1.69G [00:14<00:00, 125MB/s]


In [ ]:
!unzip 100-bird-species.zip

# 100 Birds Species Classification

## Importing Modules

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


## Preprocessing Data

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
)

In [7]:
train_set = train_datagen.flow_from_directory(directory='train/', target_size=(150, 150), batch_size=32, class_mode='binary')
valid_set = test_datagen.flow_from_directory(directory='valid/', target_size=(150, 150), batch_size=32, class_mode='binary')

Found 70626 images belonging to 450 classes.
Found 2250 images belonging to 450 classes.


In [12]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

input_tensor = Input(shape=(150, 150, 3))
model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)
x = model.output
x = Flatten()(x)
x = Dense(450, activation='softmax')(x)
model = Model(inputs=input_tensor, outputs=x)

In [13]:
for layer in model.layers[:]:
  layer.trainable = False

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 156, 156, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 75, 75, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 75, 75, 64)   256         ['conv1_conv[0][0]']         

In [22]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_set, steps_per_epoch=len(train_set), epochs=10, validation_data=valid_set, validation_steps=len(valid_set))

Epoch 1/10
 329/2208 [===>..........................] - ETA: 1:27:08 - loss: 6.4624 - accuracy: 0.0020